In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics
import random
from random import randint
import time
import os
import numpy as np
from collections import namedtuple
import cv2
from pathlib import Path
import argparse
import os
import csv
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense



# pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

In [2]:
### 실행결과 동일하게(완전 일치하지는 않음)
tf.keras.utils.set_random_seed(42)
### 텐서 연산 고정(완전 일치하지는 않음)
tf.config.experimental.enable_op_determinism()

In [3]:
folder_path_3 = "./data/이상(전도)"

In [5]:
def concat(path):
    file_lst = os.listdir(path)
    df_list = []
    for file in file_lst:
        file_name = os.path.join(path, file)
        df = pd.read_csv(file_name)
        df = df[df.NUMOFBODIES != 0].reset_index(drop=True)
###
        if len(df) < 610:
            pad_length = 610 - len(df)
            pad_df = pd.DataFrame(0, index=range(len(df), 610), columns=df.columns)
            df = pd.concat([df, pad_df])
        df['index_num'] = df.index
###
        df_list.append(df)
    #concat_df = pd.concat(df_list, ignore_index=True)
    return df_list

In [7]:
file_lst = os.listdir(folder_path_3)

In [8]:
jeon_mer = concat(folder_path_3)

In [10]:
jeon_mer

726

In [9]:
jeon_mer = pd.concat(jeon_mer, ignore_index=True)

In [10]:
jeon_mer['LABEL'] = 0#

In [11]:
full_data = jeon_mer
full_data.shape

(442860, 57)

In [12]:
full_data.head(10)

,NUMOFBODIES,ID,PELVIS_X,PELVIS_Y,PELVIS_CONFIDENCE_LEVEL,SPINE_NAVAL_X,SPINE_NAVAL_Y,SPINE_NAVA_CONFIDENCE_LEVEL,SPINE_CHEST_X,SPINE_CHEST_Y,...,ANKLE_RIGHT_X,ANKLE_RIGHT_Y,ANKLE_RIGHT_CONFIDENCE_LEVEL,HEAD_X,HEAD_Y,HEAD_CONFIDENCE_LEVEL,TIMESTAMP,FRAME_NUM,index_num,LABEL
0,1,0,1039.31100,276.14550,2,1042.32250,242.44455,2,1046.06160,214.25424,...,1025.72000,398.99350,2,1043.90590,153.86401,2,13:47:43:964,170,0,0
1,1,0,1009.34990,289.21283,2,1012.07720,255.72052,2,1014.26306,228.02060,...,993.77950,430.64490,2,1014.58670,167.23044,2,13:47:44:291,180,1,0
2,1,0,971.67100,306.22165,2,973.57420,271.92374,2,976.36865,243.25699,...,979.43980,431.56342,2,983.84845,181.48636,2,13:47:44:632,190,2,0
3,1,0,941.20825,320.63324,2,942.46760,284.51297,2,944.99854,253.97632,...,896.42633,463.96613,2,942.84375,186.55023,2,13:47:44:966,200,3,0
4,1,0,895.28510,336.34344,2,895.85330,298.78190,2,897.53076,267.26077,...,886.54040,475.30838,2,894.03860,198.99548,2,13:47:45:299,210,4,0
5,1,0,875.12890,344.10236,2,872.04450,306.42548,2,869.62305,275.13547,...,877.19960,501.52893,2,863.10583,207.23495,2,13:47:45:629,220,5,0
6,1,0,847.16003,366.58580,2,845.89210,326.25958,2,844.99950,293.15582,...,863.08746,521.89703,2,848.63810,221.82541,2,13:47:45:963,230,6,0
7,1,0,829.07100,387.64062,2,828.31390,346.82172,2,829.40350,311.83807,...,800.60190,546.36060,2,820.58057,235.94626,2,13:47:46:300,240,7,0
8,1,0,791.16030,408.65707,2,788.78705,365.56810,2,788.21173,329.15400,...,777.31040,576.61725,2,777.01120,251.17078,2,13:47:46:632,250,8,0
9,1,0,762.97670,427.49480,2,757.26434,381.51697,2,753.29030,342.49902,...,809.57904,579.15290,2,738.38890,257.66193,2,13:47:46:968,260,9,0


In [13]:
full_data = full_data.drop(full_data.columns[1:14], axis=1)
full_data.head(10)

,NUMOFBODIES,SHOULDER_LEFT_X,SHOULDER_LEFT_Y,SHOULDER_LEFT_CONFIDENCE_LEVEL,ELBOW_LEFT_X,ELBOW_LEFT_Y,ELBOW_LEFT_CONFIDENCE_LEVEL,WRIST_LEFT_X,WRIST_LEFT_Y,WRIST_LEFT_CONFIDENCE_LEVEL,...,ANKLE_RIGHT_X,ANKLE_RIGHT_Y,ANKLE_RIGHT_CONFIDENCE_LEVEL,HEAD_X,HEAD_Y,HEAD_CONFIDENCE_LEVEL,TIMESTAMP,FRAME_NUM,index_num,LABEL
0,1,1070.77920,189.47433,2,1089.56900,241.56595,2,1069.74080,281.27795,2,...,1025.72000,398.99350,2,1043.90590,153.86401,2,13:47:43:964,170,0,0
1,1,986.52295,195.07663,2,975.71490,247.72687,2,966.94450,289.43323,2,...,993.77950,430.64490,2,1014.58670,167.23044,2,13:47:44:291,180,1,0
2,1,1006.41930,222.21591,2,1010.42910,278.92624,2,1003.22350,324.54040,2,...,979.43980,431.56342,2,983.84845,181.48636,2,13:47:44:632,190,2,0
3,1,971.50397,226.08936,2,986.86554,286.76080,2,960.58563,330.07312,2,...,896.42633,463.96613,2,942.84375,186.55023,2,13:47:44:966,200,3,0
4,1,928.82150,236.77881,2,948.32715,295.47300,2,949.59020,346.13385,2,...,886.54040,475.30838,2,894.03860,198.99548,2,13:47:45:299,210,4,0
5,1,834.23160,246.88522,2,825.45430,305.26720,2,817.94763,351.40118,2,...,877.19960,501.52893,2,863.10583,207.23495,2,13:47:45:629,220,5,0
6,1,811.01710,255.24920,2,805.51170,316.70470,2,819.83075,357.17975,2,...,863.08746,521.89703,2,848.63810,221.82541,2,13:47:45:963,230,6,0
7,1,861.20980,279.48540,2,880.23210,348.52048,2,868.66656,404.65070,2,...,800.60190,546.36060,2,820.58057,235.94626,2,13:47:46:300,240,7,0
8,1,821.52120,297.97382,2,855.50020,361.06122,2,859.84106,419.27220,2,...,777.31040,576.61725,2,777.01120,251.17078,2,13:47:46:632,250,8,0
9,1,701.26940,309.55713,2,697.15330,382.70978,2,693.41320,439.99854,2,...,809.57904,579.15290,2,738.38890,257.66193,2,13:47:46:968,260,9,0


In [80]:
data_without_label = full_data.drop(['LABEL'], axis=1)

In [81]:
data_without_label.shape

(28670, 28)

In [76]:
sequence_start_indices = full_data.index[full_data['index_num'] == 0].tolist()
sequence_lengths = [sequence_start_indices[i] - sequence_start_indices[i-1] for i in range(1, len(sequence_start_indices))]
sequence_lengths.insert(0, sequence_start_indices[0])

# Min sequence length
min_sequence_length = min(sequence_lengths)

In [82]:
min_sequence_length = 610

In [83]:
X_sequences = []

# Iterating over the start indices to create sequences
for i in range(len(sequence_start_indices) - 1):
    start_index = sequence_start_indices[i]
    end_index = sequence_start_indices[i + 1]

    # Ensure the sequence has the expected length before appending
    if end_index - start_index == min_sequence_length:
        X_sequence = data_without_label.iloc[start_index:end_index].values
        X_sequences.append(X_sequence)

# Converting lists to numpy arrays
X_sequences = np.array(X_sequences)

X_sequences.shape


(46, 610, 28)

In [16]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X_sequences, y_sequences, test_size=0.3, random_state=48)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=48)

In [17]:
## 오버샘플링 (SMOTE)
#  - 2차원 이하 차원만 가능

# 학습데이터 차원 낮추기 (3차원 -> 2차원)
# X_train.shape -> (4303, 610, 32)
X_train = X_train.reshape(4303, 610 * 28)

# SMOTE를 적용할 때는 반드시 !!학습 데이터!!만 오버샘플링해야 함
smote = SMOTE(random_state=48)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

X_smote.shape, y_smote.shape

((7606, 17080), (7606,))

In [18]:
# 원래대로 차원 늘리기 (2차원 -> 3차원)
# X_smote의 변경된 데이터 개수로 바꿔주기
# - 행위마다 오버샘플링 시 데이터 개수가 다름
X_smote = X_smote.reshape(7606, 610, 28)

In [19]:
X_train.shape, X_smote.shape

((4303, 17080), (7606, 610, 28))

In [20]:
print(X_train.shape, y_train.shape)

(4303, 17080) (4303,)


In [21]:
print(X_temp.shape, y_temp.shape)

(1845, 610, 28) (1845,)


In [22]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_val = X_val.astype(np.float32)
y_val = y_val.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

X_smote = X_smote.astype(np.float32)
y_smote = y_smote.astype(np.float32)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(X_smote.shape[1], X_smote.shape[2])),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 훈련 데이터에 대한 샘플 가중치 마스크 생성
# 패딩이 아닌 값은 1, 패딩된 값은 0
sample_weight_train = np.where(X_smote != 0, 1, 0)
sample_weight_train = sample_weight_train.max(axis=-1)

# 각 시퀀스에 대한 평균 샘플 가중치 계산
sample_weight_train_avg = sample_weight_train.mean(axis=1)

In [29]:
# !!!!!행위마다 모델 이름 변경하기!!!!!
model_file_path = "./model/yugi.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(model_file_path)
# - 자동 훈련 멈추기 함수 사용 : 추가 훈련 epoch 3회, 가중치 업데이트
# - patience : 검증 손실의 최소값에 도달한 후 중단하기 전에 기다려야 하는 epoch 수
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 5,
                                        restore_best_weights=True)


# 모델 훈련
history = model.fit(X_smote, y_smote, validation_data=(X_val, y_val), epochs=20,
                    batch_size=128, sample_weight=sample_weight_train_avg,
                    callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


Epoch 1/20
60/60 [==============================] - 204s 3s/step - loss: 0.4768 - accuracy: 0.7743 - val_loss: 0.4699 - val_accuracy: 0.7243
Epoch 2/20
60/60 [==============================] - 214s 4s/step - loss: 0.4948 - accuracy: 0.7660 - val_loss: 0.5112 - val_accuracy: 0.7358
Epoch 3/20
60/60 [==============================] - 220s 4s/step - loss: 0.4780 - accuracy: 0.7750 - val_loss: 0.4333 - val_accuracy: 0.7816
Epoch 4/20
60/60 [==============================] - 220s 4s/step - loss: 0.4507 - accuracy: 0.7875 - val_loss: 0.5725 - val_accuracy: 0.6537
Epoch 5/20
60/60 [==============================] - 221s 4s/step - loss: 0.4592 - accuracy: 0.7867 - val_loss: 0.4067 - val_accuracy: 0.7853
Epoch 6/20
60/60 [==============================] - 221s 4s/step - loss: 0.4845 - accuracy: 0.7636 - val_loss: 0.4214 - val_accuracy: 0.8169
Epoch 7/20
60/60 [==============================] - 220s 4s/step - loss: 0.4845 - accuracy: 0.7633 - val_loss: 0.4565 - val_accuracy: 0.7739
Epoch 8/20
60

In [84]:
# 모델 불러오기
from keras.models import load_model
model1 = load_model('./model/jeon.h5')

In [85]:
X_sequences
X_val = X_sequences.astype(np.float32)

In [86]:
# 테스트 데이터에 대한 예측을 한 번만 수행
test_preds = model1.predict(X_val)
test_pred_idx = [np.argmax(pred) for pred in test_preds]

In [88]:
test_pred_idx

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [87]:
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# 실제 레이블과 예측 레이블을 사용하여 Confusion Matrix를 계산합니다.
conf_matrix = confusion_matrix(y_val, test_pred_idx)

# F1 Score를 계산합니다.
f1 = f1_score(y_val, test_pred_idx, average='weighted')

# Confusion Matrix를 시각화합니다.
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
# 모델에 맞게 타이틀 변경하기
plt.title('smoke smote')
plt.show()

# 결과를 출력합니다.
print("F1 Score:", f1)
print("\nClassification Report:\n", classification_report(y_val, test_pred_idx))

NameError: name 'y_val' is not defined